In [86]:
import pandas as pd
import openpyxl
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import pearsonr

In [87]:
# code = {'WHOSIS_000001':'LifeExpectancy','MDG_0000000007':'Under5_Mortality'}: LifeExpectancy data was reported every 5 year
def getDataFrame(url: str) -> pd.DataFrame:
    """
    This function aims to download data from the website throng API
    :param url: the url of the dataset
    :return: Returning data as a dataframe
    >>> getDataFrame('https://ghoapi.azureedge.net/api/WHOSIS_000002').shape[0]
    2328
    """
    data = requests.get(url).json()
    data = data['value']
    data = pd.DataFrame(data)
    return data


In [88]:
# how the health outcome change over time, but this dataset use a slightly different region categories from what the following analysis used.
def lineplot_time(HealthOutcomeData: pd.DataFrame):
    """
    :param HealthOutcomeData: data set of health outcome data: Under5_Mortality, LifeExpectancy
    :return: reformatted data set
    >>> a = [[24614321, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2015, 'SEX', 'MLE', None, None, None, None, None, None, '61.0', 61.03658, None, None, None, '2020-12-04T16:59:43.423+01:00', '2015', '2015-01-01T00:00:00+01:00', '2015-12-31T00:00:00+01:00'], [24614325, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2019, 'SEX', 'MLE', None, None, None, None, None, None, '63.3', 63.28709, None, None, None, '2020-12-04T16:59:43.533+01:00', '2019', '2019-01-01T00:00:00+01:00', '2019-12-31T00:00:00+01:00'], [24614313, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2000, 'SEX', 'MLE', None, None, None, None, None, None, '54.6', 54.57449, None, None, None, '2020-12-04T16:59:42.513+01:00', '2000', '2000-01-01T00:00:00+01:00', '2000-12-31T00:00:00+01:00'], [24614317, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2010, 'SEX', 'MLE', None, None, None, None, None, None, '59.6', 59.60036, None, None, None, '2020-12-04T16:59:43.013+01:00', '2010', '2010-01-01T00:00:00+01:00', '2010-12-31T00:00:00+01:00'], [24614329, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2000, 'SEX', 'FMLE', None, None, None, None, None, None, '55.4', 55.41726, None, None, None, '2020-12-04T16:59:43.61+01:00', '2000', '2000-01-01T00:00:00+01:00', '2000-12-31T00:00:00+01:00'], [24614333, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2010, 'SEX', 'FMLE', None, None, None, None, None, None, '60.3', 60.2972, None, None, None, '2020-12-04T16:59:43.72+01:00', '2010', '2010-01-01T00:00:00+01:00', '2010-12-31T00:00:00+01:00'], [24614337, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2015, 'SEX', 'FMLE', None, None, None, None, None, None, '62.3', 62.34584, None, None, None, '2020-12-04T16:59:43.847+01:00', '2015', '2015-01-01T00:00:00+01:00', '2015-12-31T00:00:00+01:00'], [24614341, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2019, 'SEX', 'FMLE', None, None, None, None, None, None, '63.2', 63.15551, None, None, None, '2020-12-04T16:59:43.927+01:00', '2019', '2019-01-01T00:00:00+01:00', '2019-12-31T00:00:00+01:00'], [24614345, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2000, 'SEX', 'BTSX', None, None, None, None, None, None, '55.0', 54.98949, None, None, None, '2020-12-04T16:59:44.053+01:00', '2000', '2000-01-01T00:00:00+01:00', '2000-12-31T00:00:00+01:00'], [24614349, 'WHOSIS_000001', 'COUNTRY', 'AFG', 'YEAR', 2010, 'SEX', 'BTSX', None, None, None, None, None, None, '59.9', 59.94055, None, None, None, '2020-12-04T16:59:44.18+01:00', '2010', '2010-01-01T00:00:00+01:00', '2010-12-31T00:00:00+01:00']]
    >>> cols = ['Id', 'IndicatorCode', 'SpatialDimType', 'SpatialDim', 'TimeDimType', 'TimeDim', 'Dim1Type', 'Dim1', 'Dim2Type', 'Dim2', 'Dim3Type', 'Dim3', 'DataSourceDimType', 'DataSourceDim', 'Value', 'NumericValue', 'Low', 'High', 'Comments', 'Date', 'TimeDimensionValue', 'TimeDimensionBegin', 'TimeDimensionEnd']
    >>> df = pd.DataFrame(a, columns = cols)
    >>> lineplot_time("Under5_Mortality_1")
    Traceback (most recent call last):
    AttributeError: 'str' object has no attribute 'columns'
    
    >>> lineplot_time(df[['TimeDim']])
    Traceback (most recent call last):
    KeyError

    """
    if ('TimeDim' not in HealthOutcomeData.columns) or ('NumericValue' not in HealthOutcomeData.columns) or ('SpatialDim' not in HealthOutcomeData.columns):
        raise KeyError
    b = HealthOutcomeData[(HealthOutcomeData['Dim1'] == 'BTSX') & (HealthOutcomeData['SpatialDimType'] == 'REGION')][['TimeDim','NumericValue','SpatialDim']]
    a = b[b['SpatialDim'] != 'GLOBAL'] # no global value
    a = a.astype({"TimeDim": str})
    a = a.pivot(index='TimeDim', columns='SpatialDim',values = 'NumericValue')
    a = a.rename_axis(None, axis=1)
    plt.figure(figsize=(16,8))
    p = sns.lineplot(data=a)
    p.set(xlabel = "Year", ylabel = 'Value')
    sns.set(rc={"figure.figsize":(3, 4)})
    plt.show()


In [89]:
def formatWithoutSex(df: pd.DataFrame) -> pd.DataFrame:
    """
    only applies to maternal mortality rate
    :param df: the target data set
    :return: the reformatted data set
    >>> formatWithoutSex(MaternalMortalityRatio_1)['COUNTRY']
    Traceback (most recent call last):
    KeyError: 'COUNTRY'
    """
    df = df[df['SpatialDimType'] == 'COUNTRY']
    df = df[['SpatialDim','TimeDim','NumericValue']]
    df = df.rename(columns={"SpatialDim":"countrycode", "TimeDim":"year", "NumericValue": 'MaternalMortalityRatio'})
    df = df.astype({"year": str})
    df = df[['countrycode','year','MaternalMortalityRatio']]
    return df


In [90]:
def formatWithSex(name: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    only applies to life expectancy and Under5_Mortality
    :param name: reformat with sex, pivot that column
    :param df:
    :return:
    >>> formatWithSex('Under5_Mortality', Under5_Mortality)
    Traceback (most recent call last):
    KeyError: 'SpatialDimType'
    >>> type(formatWithSex('Under5_Mortality', Under5_Mortality_1)['year'][0])
    <class 'str'>
    """
    df = df[df['SpatialDimType'] == 'COUNTRY']
    df = df[['SpatialDim','TimeDim','Dim1','NumericValue']]
    df = df.rename(columns={"SpatialDim":"countrycode", "TimeDim":"year", "Dim1":"Sex","NumericValue": 'df'})
    df = df.pivot(index=['countrycode','year'],columns = ['Sex'],values= 'df')
    df = df.reset_index()
    df.rename(columns={'BTSX':f'{name}_BTSX','FMLE':f'{name}_FMLE','MLE':f'{name}_MLE'},inplace=True)
    df = df[['countrycode','year',f'{name}_BTSX',f'{name}_FMLE',f'{name}_MLE']]
    df = df.astype({"year": str})
    df = df.rename_axis(None, axis=1) # remove "Sex" as the index name
    return df

In [91]:
def effectOfexp2015before(data: pd.DataFrame,col: str,region: str) -> tuple:
    """
    :param data: the input data set
    :param col: one of the column name needs to be tested
    :return: correlation results
    >>> effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','Europe & Central')
    (None, None)
    """
    if region not in ['East Asia & Pacific','Europe & Central Asia','Latin America & Caribbean','Middle East & North Africa','North America','South Asia','Sub-Saharan Africa']:
        return None, None
    cleaned = data.loc[(data["Year"] == 2000) | (data["Year"] == 2001) | (data["Year"] == 2002) | (data["Year"] == 2003) | (data["Year"] == 2004) | (data["Year"] == 2005) | (data["Year"] == 2006) | (data["Year"] == 2007) | (data["Year"] == 2008) | (data["Year"] == 2009) | (data["Year"] == 2010) | (data["Year"] == 2011) | (data["Year"] == 2012) | (data["Year"] == 2013) | (data["Year"] == 2014)]
    coefficient, pvalue = pearsonr(cleaned[cleaned['Region'] == region]['che_gdp'], cleaned[cleaned['Region'] == region][col])
    return coefficient, pvalue


In [92]:
# life expectancy: Europe & Central Asia positively correlated; Sub-Saharan Africa negatively correlated: differ in regions:
# conclusion from regional paper: year: before 2015 in Sub-Saharan Africa, 1 percent increase in health expenditure per capita improve life expectancy by 0.06 percent.
# in 2019: sub-Saharan Africa, 1 percent increase in health expenditure per capita improve life expectancy by 0.06 percent.
def effectOfexp2015after(data: pd.DataFrame,col: str,region: str) -> tuple:
    """
    :param data: the input data set
    :param col: one of the column name needs to be tested
    :return: correlation results
    """
    if region not in ['East Asia & Pacific','Europe & Central Asia','Latin America & Caribbean','Middle East & North Africa','North America','South Asia','Sub-Saharan Africa']:
        return None, None
    cleaned = data.loc[(data["Year"] == 2015) | (data["Year"] == 2016) | (data["Year"] == 2017) | (data["Year"] == 2018) | (data["Year"] == 2019)]
    coefficient, pvalue = pearsonr(cleaned[cleaned['Region'] == region]['che_gdp'], cleaned[cleaned['Region'] == region][col])
    return coefficient, pvalue


In [93]:
# read income data, 218 countries
incomegroup = pd.read_excel("../datasources/income group.xlsx",sheet_name='Sheet1')
healthExp = pd.read_excel("../datasources/healthExp_data.xlsx",sheet_name='cleaned')

incomegroup = pd.melt(incomegroup, id_vars=['CountryCode','CountryName'], value_vars=[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020], var_name='Year', value_name='incomeGroup')

# left join with region table.
region = pd.read_csv("../datasources/country_region.csv")[['Country','Region']]
countryInfo = incomegroup.merge(region, left_on='CountryName', right_on='Country') # inner join
countryInfo = countryInfo[['Country','CountryCode','Region','Year','incomeGroup']]
countryInfo = countryInfo.astype({"Year": str})


In [94]:
healthExp = healthExp[['country','year','che_gdp']]
healthExp = healthExp.astype({"year": str})
# merge datasets based on two columns:
countryInfoAll = countryInfo.merge(healthExp, left_on=['Country','Year'], right_on=['country','year']) # inner join
countryInfoAll = countryInfoAll[['Country','Region','CountryCode','Year','incomeGroup','che_gdp']]
countryInfoAll

,Country,Region,CountryCode,Year,incomeGroup,che_gdp
0,Afghanistan,South Asia,AFG,2000,L,NaN
1,Afghanistan,South Asia,AFG,2001,L,NaN
2,Afghanistan,South Asia,AFG,2002,L,9.443390
3,Afghanistan,South Asia,AFG,2003,L,8.941258
4,Afghanistan,South Asia,AFG,2004,L,9.808474
...,...,...,...,...,...,...
3275,Zimbabwe,Sub-Saharan Africa,ZWE,2015,L,7.452066
3276,Zimbabwe,Sub-Saharan Africa,ZWE,2016,L,7.675163
3277,Zimbabwe,Sub-Saharan Africa,ZWE,2017,L,7.469752
3278,Zimbabwe,Sub-Saharan Africa,ZWE,2018,LM,8.680062


In [95]:
countryInfoAll.Region.unique()

array(['South Asia', 'Europe & Central Asia',
       'Middle East & North Africa', 'Sub-Saharan Africa',
       'Latin America & Caribbean', 'East Asia & Pacific',
       'North America'], dtype=object)

In [96]:
countryInfoAll.isna().sum()

Country         0
Region          0
CountryCode     0
Year            0
incomeGroup     0
che_gdp        55
dtype: int64

In [97]:
LifeExpectancy_1 = getDataFrame('https://ghoapi.azureedge.net/api/WHOSIS_000001')
Under5_Mortality_1 = getDataFrame('https://ghoapi.azureedge.net/api/MDG_0000000007')
MaternalMortalityRatio_1 = getDataFrame('https://ghoapi.azureedge.net/api/MDG_0000000026')

In [ ]:
lineplot_time(LifeExpectancy_1)

In [ ]:
# Sub-Saharan Africa is part of African Region AFRO.
lineplot_time(Under5_Mortality_1)

In [ ]:
Under5_Mortality = formatWithSex('Under5_Mortality', Under5_Mortality_1)
LifeExpectancy = formatWithSex('LifeExpectancy', LifeExpectancy_1)

In [ ]:
MaternalMortalityRatio = formatWithoutSex(MaternalMortalityRatio_1)

In [ ]:
# the average of Maternal Mortality Ratio over the world was decreasing over time
plt.figure(figsize=(16,8))
MaternalMortalityRatio = MaternalMortalityRatio.sort_values('year')
sns.lineplot(data=MaternalMortalityRatio, x="year", sort=False,y="MaternalMortalityRatio")
sns.set(rc={"figure.figsize":(3, 4)})
plt.show()


In [ ]:
InfoAll = countryInfoAll.merge(LifeExpectancy, left_on=['CountryCode','Year'], right_on=['countrycode','year'], how= 'left')
InfoAll = InfoAll.merge(Under5_Mortality, left_on=['CountryCode','Year'], right_on=['countrycode','year'], how= 'left')
InfoAll = InfoAll.merge(MaternalMortalityRatio, left_on=['CountryCode','Year'], right_on=['countrycode','year'], how= 'left')
InfoAll = InfoAll[['Country','Region','CountryCode','Year','incomeGroup','che_gdp','LifeExpectancy_BTSX','LifeExpectancy_FMLE','LifeExpectancy_MLE','Under5_Mortality_BTSX','Under5_Mortality_FMLE','Under5_Mortality_MLE','MaternalMortalityRatio']]


##### all data includes health expendure, three factors, year, incomegroup, and region for each country

In [ ]:
InfoAll

##### which factor is correlated with health expenditure?

###### anova table shows that all three factors were related to health expenditure, of which the region has the most significant correlation

In [ ]:
# Ordinary Least Squares (OLS) model
model = ols('che_gdp ~ Year+incomeGroup+Region', data=InfoAll).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
# InfoAll['Year']
InfoAll = InfoAll.astype({"Year": int})
yearexp = InfoAll[['che_gdp','Year']].dropna()
pearsonr(yearexp['che_gdp'], yearexp['Year'])

In [ ]:
# health expenditure per region per year
InfoAll2015_ = InfoAll.loc[(InfoAll["Year"] == 2015) | (InfoAll["Year"] == 2016) | (InfoAll["Year"] == 2017) | (InfoAll["Year"] == 2018) | (InfoAll["Year"] == 2019)]
d = InfoAll2015_[['Region','Year','che_gdp']]
d = d.dropna()
d = d.pivot_table(index='Region', columns='Year',values = 'che_gdp', aggfunc='mean')

In [ ]:
d

###### correlation between independent var = region and dependent var = che_gdp in Year 2019


In [ ]:
# generate a boxplot to see the health expenditure distribution by income groups. Using boxplot, we can easily detect the differences between income groups

# CAN is only country in North America

plt.figure(figsize=(16,8))
ax = sns.boxplot(x='Region', y='che_gdp', data= InfoAll[InfoAll['Year'] == 2019], color='#99c2a2')
ax = sns.swarmplot(x="Region", y="che_gdp", data=  InfoAll[InfoAll['Year'] == 2019],color=".2")
ax.set_xticklabels(ax.get_xticklabels(),rotation=30, ha="right")
sns.set(rc={"figure.figsize":(3, 4)})
plt.show()
# highest value: Tuvalu in East Asia

##### Effect of Health Expenditure on Selected Health Outcomes

In [ ]:
# life Expectancy recorded data every five year. 2010, 2015, 2000, 2019.
lifeExp_clean = InfoAll[['Country','Region','CountryCode','Year','incomeGroup','che_gdp', 'LifeExpectancy_BTSX']].dropna()
# drop nan before calculating correlation
Under5_clean = InfoAll[['Country','Region','CountryCode','Year','incomeGroup','che_gdp', 'Under5_Mortality_BTSX']].dropna()
MaternalMortality_clean = InfoAll[['Country','Region','CountryCode','Year','incomeGroup','che_gdp', 'MaternalMortalityRatio']].dropna()

In [ ]:
# aligned with previous finding that in Sub-Saharan Africa Under5_Mortality in both sex decreased with health expenditure increased
print(effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','Sub-Saharan Africa'))
print(effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','Latin America & Caribbean')) #
print(effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','Europe & Central Asia')) #
print(effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','Middle East & North Africa')) #
print(effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','North America')) #
print(effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','South Asia'))
print(effectOfexp2015before(Under5_clean,'Under5_Mortality_BTSX','East Asia & Pacific'))

In [ ]:
# except for Sub-Saharan Africa and South Asia, the life expectancy increased with every increase in health expenditure
# after 2015, in Middle East & North Africa, with every one percent increase in health expenditure over total GDP, results in 0.42 percent decrease in the Under5_Mortality rate despite of sex
print(effectOfexp2015after(Under5_clean,'Under5_Mortality_BTSX','Sub-Saharan Africa'))
print(effectOfexp2015after(Under5_clean,'Under5_Mortality_BTSX','Latin America & Caribbean'))
print(effectOfexp2015after(Under5_clean,'Under5_Mortality_BTSX','Europe & Central Asia'))
print(effectOfexp2015after(Under5_clean,'Under5_Mortality_BTSX','Middle East & North Africa'))
print(effectOfexp2015after(Under5_clean,'Under5_Mortality_BTSX','North America'))
print(effectOfexp2015after(Under5_clean,'Under5_Mortality_BTSX','South Asia'))
print(effectOfexp2015after(Under5_clean,'Under5_Mortality_BTSX','East Asia & Pacific'))

In [ ]:
# only one country in north america, the relationship is weird
# except for Sub-Saharan Africa and South Asia, the life expectancy increased with every increase in health expenditure
# in Latin America & Caribbean,
# with every one percent increase in health expenditure over total GDP, results in 0.42 years increase in life expectancy
print(effectOfexp2015after(lifeExp_clean,'LifeExpectancy_BTSX','Sub-Saharan Africa'))
print(effectOfexp2015after(lifeExp_clean,'LifeExpectancy_BTSX','Latin America & Caribbean'))
print(effectOfexp2015after(lifeExp_clean,'LifeExpectancy_BTSX','Europe & Central Asia'))
print(effectOfexp2015after(lifeExp_clean,'LifeExpectancy_BTSX','Middle East & North Africa'))
print(effectOfexp2015after(lifeExp_clean,'LifeExpectancy_BTSX','North America'))
print(effectOfexp2015after(lifeExp_clean,'LifeExpectancy_BTSX','South Asia'))
print(effectOfexp2015after(lifeExp_clean,'LifeExpectancy_BTSX','East Asia & Pacific'))

In [ ]:
# maternal mortality rate for each region and its correlation with the increase/decrease in health expenditure
# except for Sub-Saharan Africa and South Asia, the mortality rate decreased with every increase in health expenditure
# in Latin America & Caribbean, with every one percent increase in health expenditure over total GDP, results in 0.35 percent decrease in the mortality rate
print(effectOfexp2015after(MaternalMortality_clean,'MaternalMortalityRatio','Sub-Saharan Africa'))
print(effectOfexp2015after(MaternalMortality_clean,'MaternalMortalityRatio','Latin America & Caribbean'))
print(effectOfexp2015after(MaternalMortality_clean,'MaternalMortalityRatio','Europe & Central Asia'))
print(effectOfexp2015after(MaternalMortality_clean,'MaternalMortalityRatio','Middle East & North Africa'))
print(effectOfexp2015after(MaternalMortality_clean,'MaternalMortalityRatio','North America'))
print(effectOfexp2015after(MaternalMortality_clean,'MaternalMortalityRatio','South Asia'))
print(effectOfexp2015after(MaternalMortality_clean,'MaternalMortalityRatio','East Asia & Pacific'))
